[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Denis-R-V/Homeworks/blob/main/Part_4/Task_1_NN.ipynb)

# Домашнее задание 1
- Написать на PyTorch forward и backward полносвязного слоя без использования autograd
- Написать 1-2 адаптивных оптимизатора
- Решить задачу нахождения корней квадратного уравнения методом градиентного спуска

In [1]:
import torch

# Задание 1
Написать на PyTorch forward и backward полносвязного слоя без использования autograd

### Функции активации ReLU, sigmoid, tanh и их backward

In [2]:
def tanh(x):                                    # гиперболический тангенс
    return 2. / (1 + torch.exp(-2*x)) - 1

def tanh_backward(da, x):                       # backward гиперболического тангенса
    tanh_ = tanh(x)
    return da * (1 - tanh_**2)    

def sigmoid(x):                                 # сигмоида
    return 1. / (1 + torch.exp(-x))

def sigmoid_backward(da, x):                    # backward сигмодиды
    sig = sigmoid(x)
    return da * sig * (1 - sig)

def relu(x):                                    # ReLU
    return torch.maximum(torch.zeros(1), x)

def relu_backward(da, x):                       # backward ReLU
    da = torch.tensor(da)
    da[x <= 0] = 0
    return da

### Функции потерь MSE и MAE и их производные

In [3]:
def mse_loss(t, y):         # функция потерь mse
    return (t - y) ** 2

def d_mse_loss(t, y):       # производная функции потерь mse
    return 2 * (y - t) 

def mae_loss(t, y):         # функция потерь mae
    return abs(t - y)

def d_mae_loss(t, y):       # производная функции потерь mae
    if t > y:
        d_mae = torch.tensor([[-1.]])
    elif t < y:
        d_mae = torch.tensor([[1.]])
    else:
        d_mae = torch.tensor([[0.]])

    return d_mae

### Полносвязный линейный слой (forward и backward полносвязного слоя без использования autograd)

In [4]:
class LinearLayer:
    def __init__(self, n_inp, n_out, activation='sigmoid'):
        
        # матрица весов модели
        self.w = torch.rand(n_out, n_inp) * 0.1
        self.b = torch.rand(n_out, 1) * 0.1
        
        # выбор активации
        if activation == 'tanh':
            self.activ = tanh        
        elif activation == 'sigmoid':
            self.activ = sigmoid
        elif activation == 'relu':
            self.activ = relu
        elif activation == 'None':
            self.activ = None
        else:
            raise Exception(f'Unknown activation "{activation}"')
        
        # служебный метод для сброса состояния слоя
        self._clear_state()

    def _clear_state(self):
        self.lin = None
        self.inp = None
        self.d_w = None
        self.d_b = None

    def forward(self, x):
        self.inp = x
        # линейное преобразование
        self.lin = self.w @ self.inp + self.b
        # применение активации
        activ = self.activ(self.lin) if self.activ is not None else self.lin
        return activ

    def backward(self, grad):
        
        # на вход поступает градиент от предыдущего слоя 
        # если слой последний, то производная функционала потерь по выходу сети
        
        if self.activ == tanh:
            grad_lin = tanh_backward(grad, self.lin) 
        elif self.activ == sigmoid:
            grad_lin = sigmoid_backward(grad, self.lin)         
        elif self.activ == relu:
            grad_lin = relu_backward(grad, self.lin)
        else:
            grad_lin = grad
        
        # градиент по весам
        m = self.inp.size()[1]
        self.d_w = (grad_lin@self.inp.t()) / m    # d_in dOut
        self.d_b = torch.sum(grad_lin, axis=1, keepdims=True) / m
        # градиент нужен для расчетов на следующем слое
        grad = self.w.t()@grad_lin

        return grad

### Модель на полносвязных линейных слоях

In [5]:
from typing import Tuple

class Model:
    # на вход подается кортеж кортежей из числа входов и выходов для каждого слоя
    def __init__(self, arch: Tuple[Tuple[int, int]], activation):
        self.layers = []
        for i, p in enumerate(arch):
            self.layers.append(LinearLayer(p[0], p[1], 
                                           # у последнего (выходного) слоя активация отключается
                                           activation=activation if i < len(arch)-1 else 'None'))
        self._clear_state()
    
    def _clear_state(self):
        for layer in self.layers:
            layer._clear_state()

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        
        return x

    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)

        return grad 

# Задание 2
Написать 1-2 адаптивных оптимизатора

### SGDMomentum

velocity = momentum (0.9-0.99)* velocity - lr*gradient  
w = w + velocity (скорость-вектор)

In [6]:
class SGDMomentum:
    def __init__(self, model: Model, lr= 0.0001, momentum=0.9):     # momentum 0.9-0.99
        self.model = model
        self.lr = lr
        self.m = momentum
        self.vel = [[torch.zeros_like(layer.w), 
                     torch.zeros_like(layer.b)] for layer in model.layers]

    def step(self):
        for i, layer in enumerate(self.model.layers):
            self.vel[i][0] = self.vel[i][0] * self.m - self.lr * layer.d_w 
            self.vel[i][1] = self.vel[i][1] * self.m - self.lr * layer.d_b 
            layer.w += self.vel[i][0]
            layer.b += self.vel[i][1]
    
    def zero_grad(self):
        self.model._clear_state()

### Adagrad

accumulated += gradient ** 2  
adapt_lr = lr / sqrt (accumulated)   
w = w - adapt_lr * gradient

In [7]:
class Adagrad:
    def __init__(self, model: Model, lr= 0.0001):
        self.model = model
        self.lr = lr
        self.accumulated = [[torch.zeros_like(layer.w), 
                             torch.zeros_like(layer.b)] for layer in model.layers]

    def step(self):
        for i, layer in enumerate(self.model.layers):
            self.accumulated[i][0] += layer.d_w ** 2
            self.accumulated[i][1] += layer.d_b ** 2
            adapt_lr_w = self.lr / torch.sqrt(self.accumulated[i][0])
            adapt_lr_b = self.lr / torch.sqrt(self.accumulated[i][1])
            layer.w -= adapt_lr_w * layer.d_w
            layer.b -= adapt_lr_b * layer.d_b
    
    def zero_grad(self):
        self.model._clear_state()

### RMSprop

accumulated += rho (0.9 - 0.99) * accumulated + (1- rho) * gradient ** 2  
adapt_lr = lr / sqrt (accumulated)   
w = w - adapt_lr * gradient

In [106]:
class RMSprop:
    def __init__(self, model: Model, rho = 0.9, lr= 0.0001):     # momentum 0.9-0.99
        self.model = model
        self.rho = rho
        self.lr = lr
        self.accumulated = [[torch.zeros_like(layer.w), 
                             torch.zeros_like(layer.b)] for layer in model.layers]

    def step(self):
        for i, layer in enumerate(self.model.layers):
            self.accumulated[i][0] += self.rho * self.accumulated[i][0] + (1 - self.rho) * layer.d_w ** 2
            self.accumulated[i][1] += self.rho * self.accumulated[i][1] + (1 - self.rho) * layer.d_b ** 2
            adapt_lr_w = self.lr / torch.sqrt(self.accumulated[i][0])
            adapt_lr_b = self.lr / torch.sqrt(self.accumulated[i][1])
            layer.w -= adapt_lr_w * layer.d_w
            layer.b -= adapt_lr_b * layer.d_b
    
    def zero_grad(self):
        self.model._clear_state()

### Adam (RMSProp + Momentum)

velocity = beta1 * velocity + (1-beta1) * gradient  
accumulated = beta2 * accumulated + (1-beta2) * gradient ** 2  
adapt_lr = lr / sqrt (accumulated)  
w = w - adapt_lr * velocity

In [9]:
class Adam:
    def __init__(self, model: Model, beta1 = 0.9, beta2 = 0.999, lr= 0.0001):     # betas=(0.9, 0.999) - в описании Adam в Torch
        self.model = model
        self.beta1 = beta1
        self.beta2 = beta2
        self.lr = lr
        self.vel = [[torch.zeros_like(layer.w), 
                     torch.zeros_like(layer.b)] for layer in model.layers]
        self.accumulated = [[torch.zeros_like(layer.w), 
                             torch.zeros_like(layer.b)] for layer in model.layers]

    def step(self):
        for i, layer in enumerate(self.model.layers):
            self.vel[i][0] = self.beta1 * self.vel[i][0] + (1 - self.beta1) * layer.d_w 
            self.vel[i][1] = self.beta1 * self.vel[i][1] + (1 - self.beta1) * layer.d_b        
            self.accumulated[i][0] = self.beta2 * self.accumulated[i][0] + (1 - self.beta2) * layer.d_w ** 2
            self.accumulated[i][1] = self.beta2 * self.accumulated[i][1] + (1 - self.beta2) * layer.d_b ** 2            
            adapt_lr_w = self.lr / torch.sqrt(self.accumulated[i][0])
            adapt_lr_b = self.lr / torch.sqrt(self.accumulated[i][1])
            layer.w -= adapt_lr_w * self.vel[i][0]
            layer.b -= adapt_lr_b * self.vel[i][1]
    
    def zero_grad(self):
        self.model._clear_state()

## Обучение модели с разными функциями активации (tanh, sigmoid, ReLU), разными функциями потерь (MSE и MAE) и разными оптимизаторами (SGDMomentum, Adagrad, RMSprop, Adam)

### Обучающая выборка

In [130]:
x = torch.distributions.uniform.Uniform(-2,2).sample([20000])
y = x**2 + torch.rand(1)*0.1

### Активация tanh, функция потерь MAE, оптимизитор SGDMomentum

In [166]:
model = Model(((1, 100), (100, 1)), activation='tanh')    # tanh, sigmois, relu или None
optim = SGDMomentum(model, lr=0.0005, momentum=0.9)                 # SGDMomentum, Adagrad, RMSprop, Adam
for e in range(20):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(torch.tensor([[val]]))
        loss = mae_loss(t, pred)                                    # mse или mae - оба работают
        grad = d_mae_loss(t, pred)                                  # mse или mae - оба работают
        model.backward(grad)
        optim.step()
        
    print(e, model.forward(torch.tensor([[1.]])), model.forward(torch.tensor([[2.]])), model.forward(torch.tensor([[-1.]])), model.forward(torch.tensor([[-2.]])))

0 tensor([[0.9672]]) tensor([[1.0646]]) tensor([[0.7711]]) tensor([[0.6726]])
1 tensor([[0.8352]]) tensor([[0.9268]]) tensor([[0.8441]]) tensor([[0.8783]])
2 tensor([[1.2830]]) tensor([[3.2077]]) tensor([[1.3370]]) tensor([[3.2094]])
3 tensor([[1.2677]]) tensor([[3.3265]]) tensor([[1.4542]]) tensor([[3.5071]])
4 tensor([[1.1807]]) tensor([[3.2911]]) tensor([[1.2423]]) tensor([[3.2880]])
5 tensor([[1.2785]]) tensor([[3.4021]]) tensor([[1.1796]]) tensor([[3.1688]])
6 tensor([[1.1411]]) tensor([[3.4381]]) tensor([[1.2435]]) tensor([[3.4794]])
7 tensor([[0.8749]]) tensor([[3.7161]]) tensor([[0.9626]]) tensor([[4.0088]])
8 tensor([[1.0547]]) tensor([[3.9575]]) tensor([[1.0259]]) tensor([[3.9202]])
9 tensor([[0.9643]]) tensor([[3.8527]]) tensor([[0.9639]]) tensor([[3.8698]])
10 tensor([[0.9815]]) tensor([[3.8546]]) tensor([[1.1287]]) tensor([[4.2059]])
11 tensor([[0.9963]]) tensor([[3.9139]]) tensor([[1.0312]]) tensor([[3.9655]])
12 tensor([[1.0468]]) tensor([[3.9190]]) tensor([[1.0996]]) te

### Активация ReLU, функция потерь MSE, оптимизитор SGDMomentum

In [180]:
model = Model(((1, 100), (100, 1)), activation='relu')    # tanh, sigmois, relu или None
optim = SGDMomentum(model, lr=0.0001, momentum=0.9)                 # SGDMomentum, Adagrad, RMSprop, Adam
for e in range(20):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(torch.tensor([[val]]))
        loss = mse_loss(t, pred)                                    # mse или mae - оба работают
        grad = d_mse_loss(t, pred)                                  # mse или mae - оба работают
        model.backward(grad)
        optim.step()
        
    print(e, model.forward(torch.tensor([[1.]])), model.forward(torch.tensor([[2.]])), model.forward(torch.tensor([[-1.]])), model.forward(torch.tensor([[-2.]])))

C:\Users\User\AppData\Local\Temp\ipykernel_22040\3544688111.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  da = torch.tensor(da)


0 tensor([[1.0413]]) tensor([[3.9128]]) tensor([[1.0412]]) tensor([[3.9068]])
1 tensor([[1.0427]]) tensor([[3.9677]]) tensor([[1.0394]]) tensor([[3.9611]])
2 tensor([[1.0416]]) tensor([[3.9876]]) tensor([[1.0375]]) tensor([[3.9827]])
3 tensor([[1.0414]]) tensor([[3.9986]]) tensor([[1.0374]]) tensor([[3.9949]])
4 tensor([[1.0414]]) tensor([[4.0050]]) tensor([[1.0389]]) tensor([[4.0024]])
5 tensor([[1.0413]]) tensor([[4.0091]]) tensor([[1.0396]]) tensor([[4.0069]])
6 tensor([[1.0412]]) tensor([[4.0123]]) tensor([[1.0403]]) tensor([[4.0103]])
7 tensor([[1.0411]]) tensor([[4.0149]]) tensor([[1.0409]]) tensor([[4.0129]])
8 tensor([[1.0410]]) tensor([[4.0171]]) tensor([[1.0413]]) tensor([[4.0150]])
9 tensor([[1.0409]]) tensor([[4.0190]]) tensor([[1.0417]]) tensor([[4.0168]])
10 tensor([[1.0408]]) tensor([[4.0207]]) tensor([[1.0420]]) tensor([[4.0183]])
11 tensor([[1.0407]]) tensor([[4.0221]]) tensor([[1.0422]]) tensor([[4.0196]])
12 tensor([[1.0408]]) tensor([[4.0232]]) tensor([[1.0424]]) te

### Активация sigmoid, функция потерь MAE, оптимизитор Adagrad

In [172]:
model = Model(((1, 100), (100, 1)), activation='sigmoid')  # tanh, sigmois, relu или None
optim = Adagrad(model, lr=0.05)                         # SGDMomentum, Adagrad, RMSprop, Adam
for e in range(20):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(torch.tensor([[val]]))
        loss = mae_loss(t, pred)                                    # mse или mae - оба работают
        grad = d_mae_loss(t, pred)                                  # mse или mae - оба работают
        model.backward(grad)
        optim.step()
        
    print(e, model.forward(torch.tensor([[1.]])), model.forward(torch.tensor([[2.]])), model.forward(torch.tensor([[-1.]])), model.forward(torch.tensor([[-2.]])))

0 tensor([[1.1748]]) tensor([[1.7752]]) tensor([[0.5272]]) tensor([[0.6050]])
1 tensor([[1.0405]]) tensor([[3.4056]]) tensor([[1.1139]]) tensor([[2.9841]])
2 tensor([[1.0395]]) tensor([[3.6545]]) tensor([[1.0455]]) tensor([[3.6359]])
3 tensor([[1.0184]]) tensor([[3.7868]]) tensor([[1.0257]]) tensor([[3.8107]])
4 tensor([[1.0227]]) tensor([[3.8413]]) tensor([[1.0294]]) tensor([[3.8718]])
5 tensor([[1.0282]]) tensor([[3.8852]]) tensor([[1.0254]]) tensor([[3.8943]])
6 tensor([[1.0298]]) tensor([[3.9039]]) tensor([[1.0322]]) tensor([[3.9299]])
7 tensor([[1.0302]]) tensor([[3.9181]]) tensor([[1.0327]]) tensor([[3.9463]])
8 tensor([[1.0299]]) tensor([[3.9293]]) tensor([[1.0320]]) tensor([[3.9560]])
9 tensor([[1.0342]]) tensor([[3.9386]]) tensor([[1.0399]]) tensor([[3.9738]])
10 tensor([[1.0327]]) tensor([[3.9455]]) tensor([[1.0369]]) tensor([[3.9738]])
11 tensor([[1.0366]]) tensor([[3.9526]]) tensor([[1.0375]]) tensor([[3.9778]])
12 tensor([[1.0351]]) tensor([[3.9581]]) tensor([[1.0375]]) te

### Активация sigmoid, функция потерь MSE, оптимизитор RMSprop

In [141]:
model = Model(((1, 100), (100, 1)), activation='sigmoid')   # tanh, sigmois, relu или None
optim = RMSprop(model, rho = 0.0001, lr=0.15)               # SGDMomentum, Adagrad, RMSprop, Adam
for e in range(20):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(torch.tensor([[val]]))
        loss = mse_loss(t, pred)                        # mse или mae - оба работают
        grad = d_mse_loss(t, pred)                      # mse или mae - оба работают
        model.backward(grad)
        optim.step()
        
    print(e, model.forward(torch.tensor([[1.]])), model.forward(torch.tensor([[2.]])), model.forward(torch.tensor([[-1.]])), model.forward(torch.tensor([[-2.]])))

0 tensor([[1.0570]]) tensor([[3.7821]]) tensor([[1.0742]]) tensor([[3.7731]])
1 tensor([[1.0479]]) tensor([[3.8381]]) tensor([[1.0546]]) tensor([[3.8344]])
2 tensor([[1.0462]]) tensor([[3.8550]]) tensor([[1.0508]]) tensor([[3.8521]])
3 tensor([[1.0446]]) tensor([[3.8593]]) tensor([[1.0494]]) tensor([[3.8584]])
4 tensor([[1.0434]]) tensor([[3.8596]]) tensor([[1.0490]]) tensor([[3.8606]])
5 tensor([[1.0434]]) tensor([[3.8600]]) tensor([[1.0493]]) tensor([[3.8621]])
6 tensor([[1.0433]]) tensor([[3.8599]]) tensor([[1.0493]]) tensor([[3.8623]])
7 tensor([[1.0430]]) tensor([[3.8595]]) tensor([[1.0492]]) tensor([[3.8622]])
8 tensor([[1.0429]]) tensor([[3.8594]]) tensor([[1.0492]]) tensor([[3.8623]])
9 tensor([[1.0430]]) tensor([[3.8597]]) tensor([[1.0494]]) tensor([[3.8628]])
10 tensor([[1.0431]]) tensor([[3.8599]]) tensor([[1.0495]]) tensor([[3.8630]])
11 tensor([[1.0431]]) tensor([[3.8600]]) tensor([[1.0495]]) tensor([[3.8630]])
12 tensor([[1.0432]]) tensor([[3.8600]]) tensor([[1.0495]]) te

В данном случае RMSProp работает только с очень маленьким rho, схождение слабое

### Активация sigmoid, функция потерь MSE, оптимизитор Adam

In [143]:
model = Model(((1, 100), (100, 1)), activation='sigmoid')  # tanh, sigmois, relu или None
#model = Model(((1, 10), (10, 100), (100, 1)), activation='relu')  # tanh, sigmois, relu или None
#optim = SGDMomentum(model, lr=0.0001, momentum=0.9)
#optim = Adagrad(model, lr=0.05)
#optim = RMSprop(model, rho = 0.9, lr=0.05)
optim = Adam(model, beta1 = 0.9, beta2 = 0.999, lr= 0.001)
for e in range(20):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(torch.tensor([[val]]))
        loss = mse_loss(t, pred)                    # mse или mae - оба работают
        grad = d_mse_loss(t, pred)                  # mse или mae - оба работают
        model.backward(grad)
        optim.step()
        
    print(e, model.forward(torch.tensor([[1.]])), model.forward(torch.tensor([[2.]])), model.forward(torch.tensor([[-1.]])), model.forward(torch.tensor([[-2.]])))

0 tensor([[1.3867]]) tensor([[3.2525]]) tensor([[1.1588]]) tensor([[3.3025]])
1 tensor([[1.0493]]) tensor([[3.7973]]) tensor([[1.0471]]) tensor([[3.8201]])
2 tensor([[1.0117]]) tensor([[3.9582]]) tensor([[1.0308]]) tensor([[3.9810]])
3 tensor([[1.0151]]) tensor([[3.9792]]) tensor([[1.0319]]) tensor([[4.0015]])
4 tensor([[1.0199]]) tensor([[3.9878]]) tensor([[1.0332]]) tensor([[4.0065]])
5 tensor([[1.0236]]) tensor([[3.9949]]) tensor([[1.0346]]) tensor([[4.0108]])
6 tensor([[1.0270]]) tensor([[4.0018]]) tensor([[1.0361]]) tensor([[4.0144]])
7 tensor([[1.0305]]) tensor([[4.0089]]) tensor([[1.0378]]) tensor([[4.0179]])
8 tensor([[1.0337]]) tensor([[4.0157]]) tensor([[1.0394]]) tensor([[4.0214]])
9 tensor([[1.0360]]) tensor([[4.0210]]) tensor([[1.0408]]) tensor([[4.0247]])
10 tensor([[1.0367]]) tensor([[4.0239]]) tensor([[1.0416]]) tensor([[4.0270]])
11 tensor([[1.0368]]) tensor([[4.0254]]) tensor([[1.0420]]) tensor([[4.0286]])
12 tensor([[1.0368]]) tensor([[4.0265]]) tensor([[1.0423]]) te

Оптимизатор Adam показал наилучший результат

# Задание 3
Решить задачу нахождения корней квадратного уравнения методом градиентного спуска

In [184]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

## Уравнение
$ x^2 - 6x + 4 = 0 $

In [181]:
# возвести в квадрат
# посчитать производную
# надо начать движение от начальной точки в направлении антиградиента с заданным шагом
# x = x - lr * grad(x)
# всегда ли сойдемся за приемлемое количество шагов?
# важна ли начальная точка?
# как найти второй корень?
# как вляет ЛР?

Посмотрим, в каких точках график равен 0

In [185]:
x = np.linspace(-1, 7, 100)
y = x ** 2 - 6 * x + 4          # 9 10
df = pd.DataFrame(data=y, index=x, columns=['function'])
df['line'] = 0
fig = px.line(df,
              labels={'index':'x', 'value':'f(x)', 'variable':'plot'},
              title="График функции f(x) = x<sup>2</sup> - 6x + 4 и линии, пересекающей ее в точках y=0")
fig.show()

Возведем уравнение в квадрат, так как ищем не минимум, а производную (0 будет минимумом функции)
$ (x^2 - 6x + 4)^2 = 0 \Rightarrow x^4 - 12x^3 + 44x^2 - 48x + 16 = 0 $  

In [186]:
x = np.linspace(-1, 7, 100)
y = (x**2 - 6*x + 4)**2     # или (x**2 - 6*x + 9)**2 (1 корень) или (x**2 - 6*x + 10)**2 (нет корней)
df = pd.DataFrame(data=y, index=x, columns=['function'])
df['line'] = 0
fig = px.line(df,
              labels={'index':'x', 'value':'f(x)', 'variable':'plot'},
              title="График функции f(x) = (x<sup>2</sup> - 6x + 4)<sup>2</sup> и линии, пересекающей ее в точках y=0")
fig.show()

В точках, в которых производная равна нулю у функции будут экстремумы (в данном случае минимумы, в которых функция равна нулю)

Посчитаем производную  

$\frac{df(x)}{dx} = (x^4 - 12x^3 + 44x^2 - 48x + 16)^{'} = (x^4)^{'} - (12x^3)^{'} + (44x^2)^{'} - (48x)^{'} + (16)^{'} = 4x^3 - 36x^2 + 88x - 48 $

## Быстрый вариант решения

In [198]:
# функция градиентного спуска
def gradient_descent(x, lr=0.001):
    grad_x = 4*x**3 - 36*x**2 + 88*x - 48
    print(f"grad x = {grad_x}")
    x = x - lr * grad_x
    print(f"x = {x}")
    return x


In [241]:
# выбираем начальную точку
x = 0

In [293]:
# выполняем шаги градиентного спуска, пока градиент не станет равен нулю или не начнет меняться x
x = gradient_descent(x, lr = 0.01)

grad x = -1.8833645754057216e-10
x = 0.7639320224973853


In [294]:
# проверим корень уравнения
y = x**2 - 6*x + 4
y

1.2633449841814581e-11

Выводы:
- для нахожденся двух корней нужно стартовать из разных точек;
- если неудачно выбрать начальные точки, то, начав градиентный спуск из разных точек, можно прийти к одному локальному минимуму;
- если выбрать большой LR, то градиент не сойдется, если маленький, то шагов будет очень много.

## Вариант решения для любого квадратного уравнения с визуализацией

Напишем библиотеку по поиску x, в которых производная квадрата функции равна нулю (около нуля) - эти x будут корнями уравнения

In [295]:
class SolverQuad:
    def __init__(self, func='(x**2)-(6*x)+4', deriv_sq='4*(x**3)-36*(x**2)+88*x-48', x1_start=-10, x2_start=10):
        self.func = func            # уравнение (функция), для которого ищем корни
        self.deriv_sq = deriv_sq    # производная квадрата функции
        self.x1 = x1_start          # точка x1, с которой начинаем градиентный спуск
        self.x2 = x2_start          # точка x2, с которой начинаем градиентный спуск
        self.prec = 1e-6            # значение градиента, которое считаем нулем

        # для графиков и выводов
        self.x1_iters = 0           # текущее количество итераций градиентного спуска для x1
        self.x2_iters = 0           # текущее количество итераций градиентного спуска для x2
        self.x1_grad_dict = {}      # значения x1, полученные в ходе градиентного спуска для построения графика
        self.x2_grad_dict = {}      # значения x2, полученные в ходе градиентного спуска для построения графика


    def grad_x1(self, x1_start=None, LR=0.001, iters=1):
        ''' Функция градиентного спуска для x1 '''
        
        # если задано стартовое значение x1 - устанавливаем текущий x1 и очищаем словарь значений градиента
        if x1_start or x1_start==0:
            self.x1 = x1_start
            self.x1_iters = 0
            self.x1_grad_dict = {}
        
        # добавляем стартовое для цикла x1 и значение квадрата функции в этой точке в словарь
        x = self.x1
        self.x1_grad_dict[x] = (eval(self.func))**2
        # считаем градиент для стартового x1 в цикле
        x1_grad = eval(self.deriv_sq)

        for i in range(iters):
            # если значение градиента в текущем x1 больше нуля (с заданной точностью)
            if abs(x1_grad) > self.prec:
                self.x1_iters += 1                              # увеличиваем счетчик итераций
                self.x1 -= LR*x1_grad                           # обновляем текущий x1 (вычитаем значение градиента, умноженного на learning rate)
                x = self.x1
                self.x1_grad_dict[x] = (eval(self.func))**2      # добавляем текущий x1 и значение квадрата функции в этой точке в словарь
                x1_grad = eval(self.deriv_sq)                   # считаем градиент для текущего x1
            # если значение градиента в текущем x1 равно нуля (меньше нуля с заданной точностью) - прерываем цикл
            else:
                break
        
        # Строим график квадрата функции с указанием точек x1, полученных в ходе градиентного спуска
        self.Visual(self.x1_grad_dict)

        # Если значение градиента равно заданному нуля - корень уравнения найден
        if abs(x1_grad) < self.prec:
            x = self.x1
            f_x1 = eval(self.func)
            if abs(f_x1) <= self.prec:
                print(f"Корень квадратного уравнения x1 найден за {self.x1_iters} циклов (задана точность градиента {self.prec}). Значение f(x) в этой точке {f_x1}")
            else:
                print(f"Найден локальный минимум x1 за {self.x1_iters} циклов (задана точность градиента {self.prec}), который не является корнем уравнения. Значение f(x) в этой точке {f_x1}")
        else:
            print(f"Выполнено {self.x1_iters} циклов градиентного спуска - корень еще не найден (задана точность градиента {self.prec})")
        return print(f"x1 = {self.x1}; gradient x1 = {x1_grad}")    


    def grad_x2(self, x2_start=None, LR=0.001, iters=1):
        ''' Функция градиентного спуска для x2 '''

        # если задано стартовое значение x2 - устанавливаем текущий x2 и очищаем словарь значений градиента
        if x2_start or x2_start==0:
            self.x2 = x2_start
            self.x2_iters = 0
            self.x2_grad_dict = {}
        
        # добавляем стартовое для цикла x2 и значение квадрата функции в этой точке в словарь
        x = self.x2
        self.x2_grad_dict[x] = (eval(self.func))**2
        # считаем градиент для стартового x2 в цикле
        x2_grad = eval(self.deriv_sq)

        for i in range(iters):
            # если значение градиента в текущем x2 больше нуля (с заданной точностью)
            if abs(x2_grad) > self.prec:
                self.x2_iters += 1                              # увеличиваем счетчик итераций
                self.x2 -= LR*x2_grad                           # обновляем текущий x2 (вычитаем значение градиента, умноженного на learning rate)
                x = self.x2
                self.x2_grad_dict[x] = (eval(self.func))**2      # добавляем текущий x2 и значение квадрата функции в этой точке в словарь
                x2_grad = eval(self.deriv_sq)                   # считаем градиент для текущего x2
            # если значение градиента в текущем x2 равно нуля (меньше нуля с заданной точностью) - прерываем цикл
            else:
                break
        
        # Строим график квадрата функции с указанием точек x2, полученных в ходе градиентного спуска
        self.Visual(self.x2_grad_dict)

        # Если значение градиента равно заданному нулю - найден минумум квадрата функции
        if abs(x2_grad) < self.prec:
            x = self.x2
            f_x2 = eval(self.func)
            if abs(f_x2) <= self.prec:
                print(f"Корень квадратного уравнения x2 найден за {self.x2_iters} циклов (задана точность градиента {self.prec}). Значение f(x) в этой точке {f_x2}")
            else:
                print(f"Найден локальный минимум x2 за {self.x2_iters} циклов (задана точность градиента {self.prec}), который не является корнем уравнения. Значение f(x) в этой точке {f_x2}")
        else:
            print(f"Выполнено {self.x2_iters} циклов градиентного спуска - корень еще не найден (задана точность градиента {self.prec})")
        return print(f"x1 = {self.x2}; gradient x2 = {x2_grad}")    


    def Visual(self, x_grad_dict):
        ''' Функция построения графика квадрата функции с указанием точек x, полученных в ходе градиентного спуска ''' 
 
        # берем из словаря, поданного на вход, ключи как x и значения как y для построения точек x, полученных в ходе градиентного спуска
        x_grad=list(x_grad_dict.keys())
        y_grad=list(x_grad_dict.values())

        # берем диапазон x для построения графика квадрата функции, привязанный к полученным в ходе градиентного спуска точкам x, считаем y
        x_grad_min = min(x_grad)
        x_grad_max = max(x_grad)
        x_grad_len = max(x_grad) - min(x_grad)
        x_func = np.linspace(x_grad_min - x_grad_len/4, x_grad_max + x_grad_len/4, 100)
        x = x_func
        y = (eval(self.func))**2
        y_func = y

        # Строим график квадрата функции с указанием точек x, полученных в ходе градиентного спуска
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x_func, y=x_func*0, mode='lines',  name='f(x)=0', marker=dict(color='DarkGray')))
        fig.add_trace(go.Scatter(x=x_func, y=y_func, mode='lines',  name='f(x)=x<sup>2</sup>', marker=dict(color='Blue')))
        fig.add_trace(go.Scatter(x=x_grad, y=y_grad, mode='markers', name='gradient descent', marker=dict(color='Pink', size=8, line=dict(color='Red', width=1.5))))
        fig.update_layout(legend_orientation="h",
                          legend=dict(x=.5, xanchor="center"),
                          margin=dict(l=0, r=0, t=0, b=0))
        fig.show()
        return

        
    def test(self):
        
        x1 = self.x1
        x2 = self.x2
        x = self.x1
        x1_grad = eval(self.deriv_sq)
        f_x1 = eval(self.func)
        x = self.x2
        x2_grad = eval(self.deriv_sq)
        f_x2 = eval(self.func)

        # Строим график квадрата функции с указанием точек x1 и x2, полученных в ходе градиентного спуска
        
        if len(self.x1_grad_dict) != 0 or len(self.x2_grad_dict) != 0:
            grad_dict = {**self.x1_grad_dict, **self.x2_grad_dict}
            self.Visual(grad_dict)

        # Для x1 и x2 градиент не сошелся
        if (abs(x1_grad) >= self.prec) and (abs(x2_grad) >= self.prec):
            print("Для x1 и x2 не завершен градиентный спуск")

        # Для x1 и/или x2 градиент сошелся 
        else:
            
            # x1 не равен x2
            if abs(abs(x1) - abs(x2)) >= self.prec:
                
                # Для x1 градиент сошелся 
                if abs(x1_grad) <= self.prec:
                    # Значение функции для x1 равно 0
                    if abs(f_x1) <= self.prec:
                        print(f"Корень квадратного уравнения x1 равен {x1:.6f}, f(x1) = {f_x1:.6f}")
                    # Значение функции для x1 не равно 0
                    else:
                        print(f"Найден локальный минимум в точке {x1:.6f}, который не является корнем уравнения, f(x1) = {f_x1:.6f}")
                # Для x1 градиент несошелся 
                else:
                    print("Для x1 не завершен градиентный спуск")
                
                # Для x2 градиент сошелся
                if abs(x2_grad) <= self.prec:
                    # Значение функции для x2 равно 0
                    if abs(f_x2) <= self.prec:
                        print(f"Корень квадратного уравнения x2 равен {x2:.6f}, f(x1) = {f_x2:.6f}")
                    # Значение функции для x2 не равно 0
                    else:
                        print(f"Найден локальный минимум в точке {x2:.6f}, который не является корнем уравнения, f(x2) = {f_x2:.6f}")
                # Для x2 градиент несошелся 
                else:
                    print("Для x2 не завершен градиентный спуск")             
            
            # x1 равен x2
            else:
                if abs(f_x1) <= self.prec:
                    x = self.x1
                    print(f"У квадратного уравнения 1 корень, который равен {self.x1:.6f}, f(x) = {eval(self.func):.6f}") 
                elif abs(f_x2) <= self.prec:
                    x = self.x2
                    print(f"У квадратного уравнения 1 корень, который равен {self.x2:.6f}, f(x) = {eval(self.func):.6f}")
                else:
                    print(f"Найден локальный минимум в точке {x1:.6f}, который не является корнем уравнения, f(x) = {f_x1:.6f}")

Инициализуруем класс (на вход подаем функцию, ее производную, опционально стартовые точки для поиска корней уравнения)

In [296]:
solv = SolverQuad(func='(x**2)-(6*x)+4', deriv_sq='4*x**3 - 36*x**2 + 88*x - 48', x1_start=0, x2_start=6)

Подбираем x1 методом градиентного спуска -  на графике можно наблюдать, как x стремится к локальному минимуму

In [297]:
solv.grad_x1(x1_start=None, LR= 0.004, iters=100)

Корень квадратного уравнения x1 найден за 96 циклов (задана точность градиента 1e-06). Значение f(x) в этой точке 1.0961733076797486e-07
x1 = 0.7639319979890301; gradient x1 = -9.804472256291774e-07



Подбираем x2 методом градиентного спуска

In [298]:
solv.grad_x2(x2_start=None, LR= 0.004, iters=100)

Корень квадратного уравнения x2 найден за 96 циклов (задана точность градиента 1e-06). Значение f(x) в этой точке 1.0961733210024249e-07
x1 = 5.2360680020109704; gradient x2 = 9.804473393160151e-07


In [299]:
solv.test()

Корень квадратного уравнения x1 равен 0.763932, f(x1) = 0.000000
Корень квадратного уравнения x2 равен 5.236068, f(x1) = 0.000000


## Функция с 1 корнем

In [324]:
solv = SolverQuad(func='(x**2)-(6*x)+9', deriv_sq='4*(x**3)-24*(x**2)+54*x-54', x1_start=0, x2_start=6)

In [325]:
solv.grad_x1(x1_start=None, LR= 0.01, iters=100)

Корень квадратного уравнения x1 найден за 99 циклов (задана точность градиента 1e-06). Значение f(x) в этой точке 1.7763568394002505e-15
x1 = 2.999999950718643; gradient x1 = -8.870643881664364e-07


In [326]:
solv.grad_x2(x2_start=None, LR= 0.01, iters=100)

Корень квадратного уравнения x2 найден за 78 циклов (задана точность градиента 1e-06). Значение f(x) в этой точке 3.552713678800501e-15
x1 = 3.0000000548772476; gradient x2 = 9.87790485851292e-07


In [327]:
solv.test()

У квадратного уравнения 1 корень, который равен 3.000000, f(x) = 0.000000


## Функция без корней

In [394]:
solv = SolverQuad(func='(x**2)-(6*x)+10', deriv_sq='4*(x**3)-36*(x**2)+112*x-120', x1_start=0, x2_start=6)

In [398]:
solv.grad_x1(x1_start=None, LR= 0.023, iters=100)

Найден локальный минимум x1 за 144 циклов (задана точность градиента 1e-06), который не является корнем уравнения. Значение f(x) в этой точке 1.000000000000055
x1 = 2.9999997642541705; gradient x1 = -9.42983319873747e-07


In [397]:
solv.grad_x2(x2_start=None, LR= 0.02, iters=100)

Найден локальный минимум x2 за 176 циклов (задана точность градиента 1e-06), который не является корнем уравнения. Значение f(x) в этой точке 1.0000000000000533
x1 = 3.0000002318992967; gradient x2 = 9.275971706301789e-07


In [399]:
solv.test()

Найден локальный минимум в точке 3.000000, который не является корнем уравнения, f(x) = 1.000000
